In [1]:
cd /Users/dcox/Dropbox/InsightFellowship/Glimpse/Redacted & Subset Data/

/Users/dcox/Dropbox/InsightFellowship/Glimpse/Redacted & Subset Data


# Baseline models for Glimpse K12. 

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score, auc

## Define functions. 

In [3]:
def lin_metrics(x, y):
    ''' Prints four common metrics for evaluating regression predictions. '''
    print('Mean Absolute Error:', round(metrics.mean_absolute_error(x, y), 4))
    print('Mean Squared Error:', round(metrics.mean_squared_error(x, y), 4))
    print('Root Mean Squared Error:', round(np.sqrt(metrics.mean_squared_error(x, y)), 4))
    print('R^2 Math:,', round(r2_score(y_test, y_pred), 4))

def bin_metrics(x, y):
    '''Prints four common metrics for evaluating classification predictions.'''
    print('Accuracy:', round(metrics.accuracy_score(x, y), 4))
    print('Precision:', round(metrics.precision_score(x, y), 4))
    print('Recall:', round(recall_score(x, y), 4))
    print('ROC_AUC:,', round(roc_auc_score(x, y), 4))
    print('F1:', round(f1_score(x, y), 4))

def plot_scatter(x, y, xmin=None, xmax=None, ymin=None, ymax=None):
    '''Plots simple scatter plot of two datasets. We'll use to plot scatter of residuals.'''
    matplotlib.rc('figure', figsize=(7, 7))
    plt.scatter(y_test, y_pred, cmap='viridis', alpha=0.1)
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)
    plt.xlabel('Predicted', fontsize=20)
    plt.ylabel('Observed', fontsize=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    return plt.show()

def plot_residuals(x, y):
    '''Plot historgram of residuals.'''
    residuals = x - y
    plt.hist(residuals, bins=50, color='black')
    plt.xlabel('Residual')
    plt.ylabel('Num Observations')
    plt.show()

def plot_cm(x, y):
    cm = confusion_matrix(x, y)
    df_cm = pd.DataFrame(cm, columns=np.unique(x), index = np.unique(x))
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted'
    sns.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 20}, fmt='g')# font size

def plot_PR_curve(true, predicted):
    precision, recall, thresholds = precision_recall_curve(true, predicted)
    ns_probs = [0 for _ in range(len(true))]
    lr_f1, lr_auc = f1_score(true, predicted), auc(recall, precision)
    # summarize scores
    print('Logistic: f1=%.2f auc=%.2f' % (lr_f1, lr_auc))
    # plot the precision-recall curves
    no_skill = len(true[true==1]) / len(true)
    plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
    plt.plot(recall, precision, marker='o', label='Logistic')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.legend()
    plt.show()

# Baseline models for dfs with continuous outcome variables = linear regression. 

### Read in the data

In [ ]:
most_obs = pd.read_csv('most_obs_cont.csv')
most_feats = pd.read_csv('most_feats_cont.csv')

### Set up dfs for predictor and outcome variables 

In [ ]:
mostObs_MathOutc = most_obs['ScantronMathPostTest']
mostObs_ReadOutc = most_obs['ScantronReadingPostTest']
mostObs_pred = most_obs.drop(['ScantronMathPostTest', 'ScantronReadingPostTest'], axis=1)
mostFeats_MathOutc = most_feats['ScantronMathPostTest']
mostFeats_ReadOutc = most_feats['ScantronReadingPostTest']
mostFeats_pred = most_feats.drop(['ScantronMathPostTest', 'ScantronReadingPostTest'], axis=1)

#### Recode categorical strings with number labels and save those dictionaries

In [ ]:
# Most observations df
mostObs_predCols = list(mostObs_pred)
mostObs_predCodes = []
for i in mostObs_predCols:
    mostObs_pred[i] = mostObs_pred[i].astype('category')
    d = dict(enumerate(mostObs_pred[i].cat.categories))
    mostObs_predCodes.append(d)
    mostObs_pred[i] = mostObs_pred[i].astype('category')
    mostObs_pred[i] = mostObs_pred[i].cat.codes

# Most features df
mostFeats_predCols = list(mostFeats_pred)
mostFeats_predCodes = []
for i in mostFeats_predCols:
    mostFeats_pred[i] = mostFeats_pred[i].astype('category')
    d = dict(enumerate(mostFeats_pred[i].cat.categories))
    mostFeats_predCodes.append(d)
    mostFeats_pred[i] = mostFeats_pred[i].astype('category')
    mostFeats_pred[i] = mostFeats_pred[i].cat.codes

### Predicting outcomes for using the 'most observations' df and linear regression baseline model. 

In [ ]:
from sklearn.linear_model import LinearRegression
# Math outcomes. 
X_train, X_test, y_train, y_test = train_test_split(mostObs_pred, mostObs_MathOutc, test_size=0.20, random_state = 649)
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
lin_metrics(y_test, y_pred)
plot_scatter(y_pred, y_test, xmin=50, xmax=3500, ymin=0, ymax=3500)
plot_residuals(y_test, y_pred)

In [ ]:
# Reading outcomes. 
X_train, X_test, y_train, y_test = train_test_split(mostObs_pred, mostObs_ReadOutc, test_size=0.20, random_state = 649)
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
lin_metrics(y_test, y_pred)
plot_scatter(y_pred, y_test, xmin=50, xmax=3800, ymin=0, ymax=3800)
plot_residuals(y_test, y_pred)

### Predicting outcomes for using the 'most features' df. 

In [ ]:
# Math outcomes. 
X_train, X_test, y_train, y_test = train_test_split(mostFeats_pred, mostFeats_MathOutc, test_size=0.20, random_state = 649)
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
lin_metrics(y_test, y_pred)
plot_scatter(y_pred, y_test, xmin=1000, xmax=3500, ymin=1000, ymax=3500)
plot_residuals(y_test, y_pred)

In [ ]:
# Reading outcomes. 
X_train, X_test, y_train, y_test = train_test_split(mostFeats_pred, mostFeats_ReadOutc, test_size=0.20, random_state = 649)
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
lin_metrics(y_test, y_pred)
plot_scatter(y_pred, y_test, xmin=1500, xmax=3800, ymin=1500, ymax=3800)
plot_residuals(y_test, y_pred)

# Baseline models for dfs with binary outcome variables = logistics regression. 

### Read in data

In [ ]:
most_obs = pd.read_csv('most_obs_binary.csv')
most_feats = pd.read_csv('most_feats_binary.csv')

In [ ]:
mostObs_MathOutc = most_obs['ScantronMathPostTestBenchmarks']
mostObs_ReadOutc = most_obs['ScantronReadingPostTestBenchmarks']
mostObs_pred = most_obs.drop(['ScantronMathPostTestBenchmarks', 'ScantronReadingPostTestBenchmarks'], axis=1)
mostFeats_MathOutc = most_feats['ScantronMathPostTestBenchmarks']
mostFeats_ReadOutc = most_feats['ScantronReadingPostTestBenchmarks']
mostFeats_pred = most_feats.drop(['ScantronMathPostTestBenchmarks', 'ScantronReadingPostTestBenchmarks'], axis=1)

In [ ]:
# Most observations df
mostObs_predCols = list(mostObs_pred)
mostObs_predCodes = []
for i in mostObs_predCols:
    mostObs_pred[i] = mostObs_pred[i].astype('category')
    d = dict(enumerate(mostObs_pred[i].cat.categories))
    mostObs_predCodes.append(d)
    mostObs_pred[i] = mostObs_pred[i].astype('category')
    mostObs_pred[i] = mostObs_pred[i].cat.codes

# Most features df
mostFeats_predCols = list(mostFeats_pred)
mostFeats_predCodes = []
for i in mostFeats_predCols:
    mostFeats_pred[i] = mostFeats_pred[i].astype('category')
    d = dict(enumerate(mostFeats_pred[i].cat.categories))
    mostFeats_predCodes.append(d)
    mostFeats_pred[i] = mostFeats_pred[i].astype('category')
    mostFeats_pred[i] = mostFeats_pred[i].cat.codes

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix

### Predicting outcomes for the 'most observations ' df. 

In [ ]:
# Math outcomes. 
X_train, X_test, y_train, y_test = train_test_split(mostObs_pred, mostObs_MathOutc, test_size=0.20, random_state = 649)
classifier = LogisticRegression(solver='liblinear')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
bin_metrics(y_test, y_pred)
plot_PR_curve(y_test, y_pred)
plot_cm(y_test, y_pred)

In [ ]:
# Read outcomes. 
X_train, X_test, y_train, y_test = train_test_split(mostObs_pred, mostObs_ReadOutc, test_size=0.20, random_state = 649)
classifier = LogisticRegression(solver='liblinear')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
bin_metrics(y_test, y_pred)
plot_cm(y_test, y_pred)
plot_PR_curve(y_test, y_pred)

## Predicting outcomes for the 'most features' df. 

In [ ]:
# Math outcomes. 
X_train, X_test, y_train, y_test = train_test_split(mostFeats_pred, mostFeats_MathOutc, test_size=0.20, random_state = 649)
classifier = LogisticRegression(solver='liblinear')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
bin_metrics(y_test, y_pred)
plot_cm(y_test, y_pred)
plot_PR_curve(y_test, y_pred)

In [ ]:
# Reading outcomes. 
X_train, X_test, y_train, y_test = train_test_split(mostFeats_pred, mostFeats_ReadOutc, test_size=0.20, random_state = 649)
classifier = LogisticRegression(solver='liblinear')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
bin_metrics(y_test, y_pred)
plot_cm(y_test, y_pred)
plot_PR_curve(y_test, y_pred)